- filtraggio dataframe con funzione recuparabile su github (aggiungi time delta)-->
mantieni solo dati sp500 (che caricherà lorenzo) che hanno date coincidenti con dataframe discorsi. 
- calcolo volatilità 
- best values

In [15]:
import pandas as pd


In [16]:
def calculate_speech_durations(dfspeeches):
    """
    This function calculates the duration of each speech in dfspeeches.
    The duration is determined by counting how many rows correspond to each unique combination
    of 'date' and 'timestart'.

    Parameters:
    dfspeeches : pandas.DataFrame
        A dataframe containing ['date', 'timestart', 'speaker', 'title'] columns.

    Returns:
    pandas.DataFrame
        A dataframe containing ['date', 'timestart', 'speaker', 'title', 'duration'] columns, where 'duration'
        is the length of each speech in minutes.
    """
    # Ensure the 'date' column is in datetime format
    dfspeeches['date'] = pd.to_datetime(dfspeeches['date'])
    
    # Group by 'date', 'timestart', 'speaker', and 'title' and count the number of rows in each group
    speech_durations = dfspeeches.groupby(['date', 'timestart', 'speaker', 'title']).size().reset_index(name='duration')
    
    return speech_durations


In [17]:

# Example usage:
# Create example dataframe
dfspeeches = pd.DataFrame({
    'date': ['2022-01-02', '2022-01-02', '2022-01-02', '2022-01-04', '2022-01-04'],
    'timestart': ['10:00', '10:00', '10:00', '11:00', '11:00'],
    'speaker': ['Speaker A', 'Speaker A', 'Speaker A', 'Speaker B', 'Speaker B'],
    'title': ['Title A', 'Title A', 'Title A', 'Title B', 'Title B']
})

# Calculate speech durations
durations_df = calculate_speech_durations(dfspeeches)

print(durations_df)

        date timestart    speaker    title  duration
0 2022-01-02     10:00  Speaker A  Title A         3
1 2022-01-04     11:00  Speaker B  Title B         2


In [18]:
def filtering(dfprices, dfspeeches):
    """
    This function filters dfprices by keeping only S&P 500 data with dates
    that coincide with the ones contained in dfspeeches.

    Parameters:
    dfprices : pandas.DataFrame
        A dataframe containing ['date', 'time', 'close', 'volume'] columns.
    dfspeeches : pandas.DataFrame
        A dataframe containing ['date', 'timestart', 'speaker', 'title'] columns.

    Returns:
    pandas.DataFrame
        A filtered dataframe containing only rows from dfprices where the 'date' is present in dfspeeches.
    """
    # Ensure the 'date' columns are in datetime format
    dfprices['date'] = pd.to_datetime(dfprices['date'])
    dfspeeches['date'] = pd.to_datetime(dfspeeches['date'])
    
    # Filter dfprices to include only dates that are present in dfspeeches
    filtered_dfprices = dfprices[dfprices['date'].isin(dfspeeches['date'])]

    return filtered_dfprices


In [19]:

# Example usage:
# Create example dataframes
dfprices = pd.DataFrame({
    'date': ['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04'],
    'time': ['09:30', '09:30', '09:30', '09:30'],
    'close': [3700, 3750, 3800, 3850],
    'volume': [1000, 1500, 2000, 2500]
})

dfspeeches = pd.DataFrame({
    'date': ['2022-01-02', '2022-01-04'],
    'timestart': ['10:00', '11:00'],
    'speaker': ['Speaker A', 'Speaker B'],
    'title': ['Title A', 'Title B']
})

# Filter dfprices using the filtering function
filtered_dfprices = filtering(dfprices, dfspeeches)

print(filtered_dfprices)


        date   time  close  volume
1 2022-01-02  09:30   3750    1500
3 2022-01-04  09:30   3850    2500


In [21]:
def filtering(dfprices, dfspeeches):
    """
    This function filters dfprices by keeping only rows where the date is present in dfspeeches.
    Additionally, it considers the duration of speeches to filter rows in dfprices based on the time range.

    Parameters:
    dfprices : pandas.DataFrame
        A dataframe containing ['date', 'time', 'close', 'volume'] columns.
    dfspeeches: pandas.DataFrame
        A dataframe containing ['date', 'timestart', 'speaker', 'title'] columns.

    Returns:
    pandas.DataFrame
        A filtered dataframe containing only rows from dfprices where the 'date' and 'time' fall within the
        time range of speeches in dfspeeches.
    """
    # Ensure the 'date' and 'time' columns are in datetime format
    dfprices['date'] = pd.to_datetime(dfprices['date'])
    dfprices['time'] = pd.to_datetime(dfprices['time'], format='%H:%M').dt.time
    
    # Calculate speech durations
    speech_durations = calculate_speech_durations(dfspeeches)
    
    # Initialize an empty list to store the filtered rows
    filtered_rows = []
    
    # Iterate over each speech to filter dfprices
    for _, speech in speech_durations.iterrows():
        speech_date = speech['date']
        start_time = pd.to_datetime(speech['timestart'], format='%H:%M').time()
        duration = speech['duration']
        
        # Calculate end time based on the duration
        end_time = (pd.to_datetime(speech['timestart'], format='%H:%M') + pd.Timedelta(minutes=duration)).time()
        
        # Filter dfprices for the current speech date and time range
        mask = (dfprices['date'] == speech_date) & \
               (dfprices['time'] >= start_time) & \
               (dfprices['time'] <= end_time)
        
        filtered_rows.append(dfprices[mask])
    
    # Concatenate all filtered rows into a single dataframe
    filtered_df = pd.concat(filtered_rows)
    
    return filtered_df


In [22]:
# Example usage:
# Create example dataframes
dfprices = pd.DataFrame({
    'date': ['2022-01-02', '2022-01-02', '2022-01-02', '2022-01-02', '2022-01-02', '2022-01-04', '2022-01-04', '2022-01-04'],
    'time': ['10:00', '10:01', '10:02', '10:03', '10:04', '11:00', '11:01', '11:02'],
    'close': [100, 101, 102, 103, 104, 200, 201, 202],
    'volume': [1000, 1100, 1200, 1300, 1400, 2000, 2100, 2200]
})

dfspeeches = pd.DataFrame({
    'date': ['2022-01-03', '2022-01-03', '2022-01-03', '2022-01-04', '2022-01-04'],
    'timestart': ['10:00', '10:00', '10:00', '11:00', '11:00'],
    'speaker': ['Speaker A', 'Speaker A', 'Speaker A', 'Speaker B', 'Speaker B'],
    'title': ['Title A', 'Title A', 'Title A', 'Title B', 'Title B']
})

# Calculate speech durations
speech_durations = calculate_speech_durations(dfspeeches)

# Display speech durations
print("Speech Durations:")
print(speech_durations)

# Filter dfprices based on dfspeeches
filtered_df = filtering(dfprices, dfspeeches)

print("\nFiltered Prices:")
print(filtered_df)

Speech Durations:
        date timestart    speaker    title  duration
0 2022-01-03     10:00  Speaker A  Title A         3
1 2022-01-04     11:00  Speaker B  Title B         2

Filtered Prices:
        date      time  close  volume
5 2022-01-04  11:00:00    200    2000
6 2022-01-04  11:01:00    201    2100
7 2022-01-04  11:02:00    202    2200


In [23]:
def volatility_calculator(dfprices):
    """
    Calculate daily volatility for the 'close' column in the dataframe.

    Parameters:
    dfprices : pandas.DataFrame
        A dataframe containing ['date', 'time', 'close', 'volume'] columns.

    Returns:
    pandas.Series
        A series with dates as the index and the standard deviation
        of daily percentage changes (volatility) as the values.
    """

    # Calculate the daily percentage change in closing prices
    dfprices['pct_change'] = dfprices['close'].pct_change()
    
    # Group by 'date' and calculate the standard deviation of percentage changes
    volatility_series = dfprices.groupby('date')['pct_change'].std()
    
    # Rename the resulting series for clarity
    volatility_series.name = 'volatility'
    
    # Ensure the index is in datetime format
    volatility_series.index = pd.to_datetime(volatility_series.index)
    
    return volatility_series


In [27]:
# Example usage

dfprices = pd.DataFrame({
    'date': ['2022-01-01', '2022-01-01', '2022-01-01', '2022-01-01', '2022-01-03', '2022-01-03', '2022-01-03'],
    'close': [100, 101, 102, 105, 110, 105, 110]
})

# Calculate volatility
volatility = volatility_calculator(dfprices)
print(volatility)


date
2022-01-01    0.011236
2022-01-03    0.053736
Name: volatility, dtype: float64


In [35]:
def get_best_values(volatility, number):
    """
    Get the top dates with the highest volatility and return a dataframe with dates and volatility values.

    Parameters:
    volatility : pandas.Series
        A series with dates as the index and volatility as the values.
    number : int
        The number of top volatility dates to select.

    Returns:
    pandas.DataFrame
        A dataframe containing the top `number` dates and their corresponding volatility values.
    """
    
    # Drop NaN values in volatility to avoid issues
    volatility = volatility.dropna()

    # Sort the volatility in descending order to get the top dates with highest volatility
    top_volatility = volatility.sort_values(ascending=False).head(number)

    # Create a DataFrame with 'date' and 'volatility' columns
    result_df = top_volatility.reset_index()
    result_df.columns = ['date', 'volatility']
    
    return result_df


In [37]:
# Example DataFrame for volatility (volatility by date)
volatility = pd.Series({
    '2024-01-08': 0.000298,
    '2024-01-16': 0.002092,
    '2024-01-17': 0.000665,
    '2024-02-02': 0.005688,
    '2024-02-07': 0.001823,
    '2024-10-10': 0.000763,
    '2024-10-11': 0.00111
})

# Get the top 3 volatility dates
top_volatility_df = get_best_values(volatility, 3)

top_volatility_df


,date,volatility
0,2024-02-02,0.005688
1,2024-01-16,0.002092
2,2024-02-07,0.001823
